In [ ]:
# import ipyparallel as ipp
from multiprocessing import Pool

import EEGAnalysis as ea
import h5py
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt
import re, os
from tqdm import tqdm

tspec = np.linspace(-3,3,12000)

patient_id = 'wangjinan'
patient = ea.DataManager('/media/STORAGE/EEG/Data/').get_patient(patient_id)
_dataset = h5py.File('./%s-5-delta.h5'%patient_id, 'r')

_dates = list(_dataset.keys())
print(_dates)

# _dataset.close()

# for patient [zhouchen, shuyunfan, fansulong, wuzhenwei]
# _behavior = pd.read_csv('../../Step_3_Behavior_Stat/fansulong-behavior-final.csv', dtype={'date':'str'})
# check_behavior_prediction = lambda x,y,z: _behavior.marker[(_behavior.paradigm == x) & (_behavior['date'] == y)].values < z

# for other patients
def check_behavior_prediction(paradigm, date, thresh=0):
    _sign, _interval = re.findall(r'(\d)-(\d+)', paradigm)[0]
    
    _b_marker = patient.get_marker('behavior', file='%s-%s'%(date, _sign))
    _m_marker = patient.get_marker('marker', file='%s-%s'%(date, _sign))
    
    if _interval == '5':
        return ea.get_relative_behavior_time(_m_marker, _b_marker)[20:] < thresh
    elif _interval == '10':
        return ea.get_relative_behavior_time(_m_marker, _b_marker)[:20] < thresh
    else:
        raise ValueError('unknown paradigm: %s'%paradigm)


# _behavior[(_behavior.paradigm=='1-10') & (_behavior.date=='180901')].head(10)
# _behavior.head()

In [ ]:
# test
_prediction = check_behavior_prediction('1-10', '190330')
_prediction

In [ ]:
def _step_plot(chidx):
    _ch_itpc_with_prediction = []
    _ch_itpc_without_prediction = []

    for idate in _dates:
        _entry = np.array(_dataset[idate]['1']['%03d'%chidx])
        _prediction = check_behavior_prediction('1-5', idate, 0)

        try:  # skip when that day has no prediction and vice versa.
            _ch_itpc_with_prediction.append(_entry[_prediction])
        except IndexError:
            pass
        
        try:
            _ch_itpc_without_prediction.append(_entry[~_prediction])
        except IndexError:
            pass

    _ch_itpc_with_prediction = np.array([ea.decomposition.dwt_itpc(item.reshape((1,)+item.shape))[0] for item in _ch_itpc_with_prediction])
    _ch_itpc_without_prediction = np.array([ea.decomposition.dwt_itpc(item.reshape((1,)+item.shape))[0] for item in _ch_itpc_without_prediction])


    plt.figure(figsize=(16,6))

    plt.subplot(1,2,1)
    _curve = np.mean(_ch_itpc_with_prediction, 0)
    _sem = np.std(_ch_itpc_with_prediction, 0) / np.size(_ch_itpc_with_prediction, 0)

    plt.plot(tspec, _ch_itpc_with_prediction.T, c='k', alpha=0.1)
    plt.plot(tspec, _curve)
    plt.fill_between(tspec, _curve-_sem, _curve+_sem, alpha=0.3)
    plt.ylim((0,1))

    plt.title('w/')

    plt.subplot(1,2,2)
    _curve = np.mean(_ch_itpc_without_prediction, 0)
    _sem = np.std(_ch_itpc_without_prediction, 0) / np.size(_ch_itpc_without_prediction, 0)

    plt.plot(tspec, _ch_itpc_without_prediction.T, c='k', alpha=0.1)
    plt.plot(tspec, _curve)
    plt.fill_between(tspec, _curve-_sem, _curve+_sem, alpha=0.3)
    plt.ylim((0,1))
    plt.title('w/o')

    plt.suptitle('channel %d'%chidx)
    plt.savefig("ITPC_seq_export_behavior/itpc_prediction-5-delta/channel-%d.png"%chidx, bbox_inches="tight")
    plt.close()

In [ ]:
# preview

_step_plot(10)
# plt.savefig("ITPC_seq_export_behavior/itpc_prediction-5-delta/channel-%d.png"%chidx, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# export

subfolder = 'behavior-prediction-%s-5'%patient_id
if not os.path.isdir(os.path.join('export', subfolder)):
    ea.mktree(os.path.join('export', subfolder))
    
total_ch = len(_dataset[_dates[0]]['1'])

# def _step(chidx):
#     _step_plot(chidx)
#     plt.savefig(os.path.join('export', subfolder, "channel-%d.png"%chidx), bbox_inches="tight")
    
# with Pool(processes=5) as _p:
#     list(tqdm(_p.imap_unordered(_step_plot, range(total_ch)), total = total_ch))

for chidx in tqdm(range(total_ch)):
    _step_plot(chidx)
    plt.savefig(os.path.join('export', subfolder, "channel-%d.png"%chidx), bbox_inches="tight")
    plt.close()

In [ ]:
_dataset.close()

---

In [ ]:
_location = pd.read_csv('/media/STORAGE/EEG/electrode_layout/layout_fansulong/fansulong_tal_coord.td.fill.csv')
_location.head()

# _c = ['Brodmann area 23', 'Brodmann area 24','Brodmann area 29', 'Brodmann area 30', 'Brodmann area 31', 'Brodmann area 32', 'Brodmann area 33', ]  # Cingulate,zhouchen &shuyunfan
# _c = ['Brodmann area 17', 'Brodmann area 18', 'Brodmann area 19', ]  # Visual, zhouchen
# _c = ['Brodmann area 7', ]  # PCun,zhouchen
_c = ['Brodmann area 21']  # MTG,middle temporal gyrus, zhouchen,shuyunfan,fansulong,wuzhenwei,wangjinan
# _c = ['Brodmann area 38','Brodmann area 22']  # STG, superior temporal gyrus,zhouchen,wuzhenwei,zhangchen,wangjinan
# _c = ['Brodmann area 20']  # ITG, inferior temporal gyrus,fansulong
# _c = ['Brodmann area 35', 'Brodmann area 34', 'Brodmann area 28', 'Amygdala','Hippocampus']  # PHG, Parahippocampal gyrus
# _c = ['Brodmann area 45','Brodmann area 46','Brodmann area 47']  # IFG,inferior frontal gyrus
# _c = ['Brodmann area 3','Brodmann area 2','Brodmann area 1']  # postcentral gyrus (Primary somatosensory cortex),shuyunfan
# _c = ['Brodmann area 4']  # M1,primary motor cortex, shuyunfan
# _c = ['Brodmann area 6']  # SMA, Supplementary motor area, shuyunfan
# _c = ['Brodmann area 40']  # IPL, Inferior Parietal Lobule, wuzhenwei & zhangchen
# _c = ['Brodmann area 37']  # FG, Fusigorm gyrus, wuzhenwei
# _c = ['Brodmann area 13']  # INS, Insula,wuzhenwei & zhangchen & wangjinan
# _c = ['Brodmann area 39']  # AG,Angular gyrus,zhangchen
# _c = ['Brodmann area 41']  # AC, Auditory cortex,zhangchen
# _c = ['Brodmann area 10']  # APFC, Antirior prefrontal cortex,wangjinan
# _c = ['Brodmann area 20']  # ITL, Inferior temporal lobel,wangjinan


_target = np.ones(shape=(0,), dtype='int')
for item in _c:
    _stack = _location['Record Number'][_location['Level 5'] == item].values
    print(item, np.unique(_stack))
    _target = np.hstack((_target, _stack))
_target = np.unique(_target)
_target 

In [ ]:
%%time
_target_prediction = []
_target_noprediction = []

for chidx in _target:
    _ch_itpc_with_prediction = []
    _ch_itpc_without_prediction = []

    for idate in _dates:
        _entry = np.array(_dataset[idate]['1']['%03d'%chidx])
        _prediction = check_behavior_prediction('1-10', idate, 0)

        _ch_itpc_with_prediction.append(_entry[_prediction])
        _ch_itpc_without_prediction.append(_entry[~_prediction])

    _ch_itpc_with_prediction = np.array([ea.decomposition.dwt_itpc(item.reshape((1,)+item.shape))[0] for item in _ch_itpc_with_prediction])
    _ch_itpc_without_prediction = np.array([ea.decomposition.dwt_itpc(item.reshape((1,)+item.shape))[0] for item in _ch_itpc_without_prediction])
    
    [_target_prediction.append(item) for item in _ch_itpc_with_prediction]
    [_target_noprediction.append(item) for item in _ch_itpc_without_prediction]


In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
_raw = np.array(_target_prediction)
_curve = np.mean(_raw, 0)
_sem = np.std(_raw, 0) / np.sqrt(np.size(_raw, 0))

plt.plot(tspec, _curve)
plt.fill_between(tspec, _curve-_sem, _curve+_sem, alpha=0.5)
plt.ylim((0,1))

plt.subplot(1,2,2)
_raw = np.array(_target_noprediction)
_curve = np.mean(_raw, 0)
_sem = np.std(_raw, 0) / np.sqrt(np.size(_raw, 0))

plt.plot(tspec, _curve)
plt.fill_between(tspec, _curve-_sem, _curve+_sem, alpha=0.5)
plt.ylim((0,1))

plt.suptitle('zhouchen-cingulate')

# plt.show()
plt.savefig('ITPC_seq_export_behavior/zhouchen-cg-10.png')
plt.savefig('ITPC_seq_export_behavior/zhouchen-cg-10.eps')

In [ ]:
_raw = np.array(_target_noprediction)
# plt.plot(tspec, _raw.T, 'k', alpha=0.1)
plt.plot(np.mean(_raw, 0))
plt.ylim((0,1))
plt.show()